In [1]:
import numpy as np
import copy
import time
import pytz
import datetime
import math
import re
import os
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import seaborn as sns

# whether or not to split into train and test
split_data = True

In [2]:
def removeHeader(data):
    newData = data[1:]
    return newData
 
import matplotlib.pyplot as plt 
import pandas as pd 

def getDiff(y_pred, y_test):

    #print(y_pred[0] - float(y_test[0]))
    
    diffArray = []
    
    for i in range(len(y_test)):
        diffArray.append(y_pred[i] - float(y_test[i]))
        
    return diffArray

def plotDiff(diffArray):
    
    #print(features)
    
    labels = []
    for x in range(len(y_pred)):
        labels.append(x)
        
        
    fig = plt.figure()
    plt.scatter(labels, diffArray)
    plt.show()
    
def removeFeat(diffArray):
    greatest = 0
    greatInd = 0
    #i = 0
    
    for x in range(len(diffArray)):
        if diffArray[x] > greatest:
            greatest = diffArray[x]
            greatInd = x
        #i = i + 1
    #print(diffArray[greatInd])
    del diffArray[greatInd]
    return(diffArray)
        
    

In [3]:
from pandas import read_csv
from sklearn.model_selection import train_test_split
 
# load the dataset
def load_dataset(filename, features, disease):
    # load the dataset as a pandas DataFrame
    data = read_csv(filename, header=None)
    dataset = data.values
    
    #print(dataset.shape)
    #print(dataset[:,features])
    labels = dataset[0][features]
    
    dataset = splitByDis(dataset, disease)

    newDataset = dataset[:,features]
    # split into input (X) and output (y) variables
    
    #print(newDataset)

    newDataset = removeHeader(newDataset)
    
    #print(newDataset)

    X = newDataset[:, :-1]
    y = newDataset[:,-1]
    

    return X, y
 
# load the dataset


def getTrainTest(csv, features, disease):
    X, y = load_dataset(csv, features, disease)
    # split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)
    features = y
    
    return X_train, X_test, y_train, y_test



def convertLabels(csv, features):
    data = read_csv(csv, header=None)
    dataset = data.values
    #print(dataset.shape)
    #print(dataset[:,features])
    labels = dataset[0][features]
    
    #newDataset = dataset[:,features]
    
    print(features)
    print(labels)
    
    return labels
    
    


In [94]:
from sklearn.linear_model import LinearRegression

import pandas as pd
import numpy as np
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats
#getTrainTest('/Users/soniasharapova/Documents/FatiResearch/dummyDataset.csv', features)


def predictionsIter(csv, feats, disease, permutationNp):
    

    length = len(feats)
    
    predIters=[]
    iterated=[]
    
    for y in range(length - 1):
        featurePred=[]
        featureSubset = permutationNp[y:]
        featsCopy = featureSubset

        print("featureSubset", featureSubset)
        print("featsCopy", featsCopy)
        length = len(featureSubset)
        #predIter = predictions(csv, featureSubset, disease)
        #predIters.append(predIter)
        #print(predIters)
        
        for x in range(length-2):
            print("featsCopy[x]", featsCopy[x])
            np.append(featsCopy[:x], featsCopy[(x+1):])
            print("featsCopy", featsCopy)
            predict = predictions(csv, featsCopy, disease)
            featurePred.append(predict)
            featsCopy = []
            featsCopy = featureSubset 
        print(featureSubset)
        print(y)
        print(featurePred)
        L = [ (featurePred[i],i) for i in range(len(featurePred)) ]

        L.sort()
        sorted_l,permutation = zip(*L)
        permutationNp = np.array(permutation)
        iterated.append(permutationNp[-1])
        print("???", permutationNp) 


def predictions(csv, feats, disease):
    X_train, X_test, y_train, y_test = getTrainTest(csv, feats, disease)

    clf = LinearRegression()
    #clf = RandomForestRegressor()

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    #print(y_pred)
    
    #print(clf.coef_)

    #print(y_pred)
    #print(y_test)
    
    #print(y_pred)
    #print(np.array(y_test).astype(float))
    pred = np.linalg.norm(y_pred - np.array(y_test).astype(float))
    #print(pred)
    return(pred)


'''
def remWorstFeat(features, predictions):
    worst = 1000
    for x in range(len(predictions)):
      
        if(predictions[x] < predictions[worst]):
            worst = x
            #print(worst)
    del features[x]
    return features
'''

def splitByDis(csv, dis):
    csv = removeHeader(csv)
    csvSubset = []
    for row in csv:
        if int(row[-2]) == dis:
            csvSubset.append(row)
    csvSubset = np.array(csvSubset)
    return csvSubset
    
def bestFeats(csv, feats, disease):
    featurePred=[]
    featsCopy = feats
    #print(feats)
    #print(featsCopy)
    length = len(feats)
    
    #predictions for CF (1)
    #predictions for Healthy (2)
    #predictions for COPD (3)
    
    for x in range(length-1):
        featsCopy = featsCopy[:x] + featsCopy[(x+1):]
        predict = predictions(csv, featsCopy, disease)
        featurePred.append(predict)
        featsCopy = []
        featsCopy = feats        
    #featurePred shows the prediction if this feature was removed. If score is low, this means prediction would be lower if this was removed
    L = [ (featurePred[i],i) for i in range(len(featurePred)) ]

    L.sort()
    sorted_l,permutation = zip(*L)
    permutationNp = np.array(permutation)
    print("???", permutationNp) 

    
    predIters = predictionsIter(csv, feats, disease, permutationNp)
        
    print("!!!", predIters)
    
    print(featurePred) 
    #print(predIters) 
    return permutationNp



    
#features = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,
            #38,39,40,41,42,43,44,45,46,47]
features = [0,1,2,3,4,5,6,7]

predsDiaFCF = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combDiaFatty.csv', features, 1)
convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combDiaFatty.csv', predsDiaFCF)



??? [2 1 6 0 3 5 4]
featureSubset [2 1 6 0 3 5 4]
featsCopy [2 1 6 0 3 5 4]
featsCopy[x] 2
featsCopy [2 1 6 0 3 5 4]
featsCopy[x] 1
featsCopy [2 1 6 0 3 5 4]
featsCopy[x] 6
featsCopy [2 1 6 0 3 5 4]
featsCopy[x] 0
featsCopy [2 1 6 0 3 5 4]
featsCopy[x] 3
featsCopy [2 1 6 0 3 5 4]
[2 1 6 0 3 5 4]
0
[97.78853535257157, 97.78853535257157, 97.78853535257157, 97.78853535257157, 97.78853535257157]
??? [0 1 2 3 4]
featureSubset [1 2 3 4]
featsCopy [1 2 3 4]
featsCopy[x] 1
featsCopy [1 2 3 4]
featsCopy[x] 2
featsCopy [1 2 3 4]
[1 2 3 4]
1
[96.27759686815094, 96.27759686815094]
??? [0 1]
featureSubset []
featsCopy []
[]
2
[]


ValueError: not enough values to unpack (expected 2, got 0)

# Fatty

Comb Dia Fatty

In [72]:
predsDiaFCF = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combDiaFatty.csv', features, 1)
convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combDiaFatty.csv', predsDiaFCF)

predsDiaFHealthy = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combDiaFatty.csv', features, 2)
convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combDiaFatty.csv', predsDiaFHealthy)

predsDiaFCOPD = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combDiaFatty.csv', features, 3)
convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combDiaFatty.csv', predsDiaFCOPD)


??? [2 1 6 0 3 5 4]
[24.628339239091524, 24.45239110367443, 24.062383036945246, 24.86575645220897, 26.208563966357577, 25.04632612236095, 24.57500540566084]
[2 1 6 0 3 5 4]
['BMI' 'Age' 'Disease (CF=1,H=2,copd=3)' 'Gender(1=F)' 'Sub' 'FEV1%' 'Vis']
??? [3 6 1 0 5 2 4]
[15.019649291697355, 14.152551039580482, 17.687017573412362, 13.584549663070321, 18.868123773916416, 16.93026930469916, 13.888161872810244]
[3 6 1 0 5 2 4]
['Sub' 'Disease (CF=1,H=2,copd=3)' 'Age' 'Gender(1=F)' 'FEV1%' 'BMI' 'Vis']
??? [3 2 6 5 0 1 4]
[10.571214890378451, 10.605008075385262, 9.756012893460854, 9.417375169655266, 10.718716335771346, 10.133694242900209, 10.103990095202475]
[3 2 6 5 0 1 4]
['Sub' 'BMI' 'Disease (CF=1,H=2,copd=3)' 'FEV1%' 'Gender(1=F)' 'Age' 'Vis']


array(['Sub', 'BMI', 'Disease (CF=1,H=2,copd=3)', 'FEV1%', 'Gender(1=F)',
       'Age', 'Vis'], dtype=object)

Comb Pecs Fatty

In [6]:
predsPecsFCF = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combPecsFatty.csv', features, 1)
convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combPecsFatty.csv', predsPecsFCF)

predsPecsFHealthy = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combPecsFatty.csv', features, 2)
convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combPecsFatty.csv', predsPecsFHealthy)

predsPecsFCOPD = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combPecsFatty.csv', features, 3)
convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combPecsFatty.csv', predsPecsFCOPD)


[41.52615669576034, 45.44168661224779, 44.01572097785328, 38.47681860157886, 41.2996220381423, 32.1532829868478, 32.396620088712694, 46.091188607113104, 0.0]
[8 5 6 3 4 0 2 1 7]
['Pecs Fatty%' 'FEV1%' 'FVC % ' 'Sub' 'Vis' 'Gender(1=F)' 'BMI' 'Age'
 'Disease (CF=1,H=2,copd=3)']
[29.596295211884783, 31.392832342230346, 29.056544529248388, 40.705084584341414, 33.389884637747706, 31.846755889454666, 37.14749389358259, 42.539792018687024, 0.0]
[8 2 0 1 5 4 6 3 7]
['Pecs Fatty%' 'BMI' 'Gender(1=F)' 'Age' 'FEV1%' 'Vis' 'FVC % ' 'Sub'
 'Disease (CF=1,H=2,copd=3)']
[12.514664150828803, 31.98427970307864, 37.84560856395361, 20.035572764857346, 15.100370528748785, 12.435171657226007, 14.552950894175058, 19.744174077275577, 0.0]
[8 5 0 6 4 7 3 1 2]
['Pecs Fatty%' 'FEV1%' 'Gender(1=F)' 'FVC % ' 'Vis'
 'Disease (CF=1,H=2,copd=3)' 'Sub' 'Age' 'BMI']


Comb Para Fatty

In [7]:
predsParaFCF = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combParaFatty.csv', features, 1)
convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combParaFatty.csv', predsParaFCF)

predsParaFHealthy = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combParaFatty.csv', features, 2)
convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combParaFatty.csv', predsParaFHealthy)

predsParaFCOPD = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combParaFatty.csv', features, 3)
convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combParaFatty.csv', predsParaFCOPD)


[46.615377389732124, 34.103254728819785, 37.40779842250722, 39.80035649639739, 40.268008442870396, 33.78114081167288, 35.99417805343154, 40.47462596396224, 0.0]
[8 5 1 6 2 3 4 7 0]
['Para Fatty%' 'FEV1%' 'Age' 'FVC % ' 'BMI' 'Sub' 'Vis'
 'Disease (CF=1,H=2,copd=3)' 'Gender(1=F)']
[46.99542289447723, 49.21953229592538, 46.70383702957177, 58.3095081465515, 47.28350421481455, 45.37668049970915, 46.87041258838695, 46.785519803400156, 0.0]
[8 5 2 7 6 0 4 1 3]
['Para Fatty%' 'FEV1%' 'BMI' 'Disease (CF=1,H=2,copd=3)' 'FVC % '
 'Gender(1=F)' 'Vis' 'Age' 'Sub']
[29.82032380021011, 31.772007212021, 48.13868090533803, 37.579828107446794, 28.649009213189444, 29.07032989288526, 26.826515445832875, 46.22288797221579, 0.0]
[8 6 4 5 0 1 3 7 2]
['Para Fatty%' 'FVC % ' 'Vis' 'FEV1%' 'Gender(1=F)' 'Age' 'Sub'
 'Disease (CF=1,H=2,copd=3)' 'BMI']


Comb Abd Fatty

In [8]:
predsAbdFCF = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combAbdFatty.csv', features, 1)
convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combAbdFatty.csv', predsAbdFCF)

predsAbdFHealthy = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combAbdFatty.csv', features, 2)
convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combAbdFatty.csv', predsAbdFHealthy)

predsAbdFCOPD = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combAbdFatty.csv', features, 3)
convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combAbdFatty.csv', predsAbdFCOPD)


[41.964418726724446, 32.50883236630417, 39.87305740783789, 40.20079263644784, 40.407222207444256, 38.603804324966255, 39.86411802575015, 39.8839081425823, 0.0]
[8 1 5 6 2 7 3 4 0]
['Abd Fatty%' 'Age' 'FEV1%' 'FVC % ' 'BMI' 'Disease (CF=1,H=2,copd=3)'
 'Sub' 'Vis' 'Gender(1=F)']
[37.63738872137609, 34.80318557408691, 33.34098499820637, 32.05625654338098, 40.51301622356212, 35.42503692372685, 35.365099970555946, 35.87539527287004, 0.0]
[8 3 2 1 6 5 7 0 4]
['Abd Fatty%' 'Sub' 'BMI' 'Age' 'FVC % ' 'FEV1%'
 'Disease (CF=1,H=2,copd=3)' 'Gender(1=F)' 'Vis']
[17.62560309276487, 36.59225163980832, 22.529606192085986, 19.709209759842317, 28.649092847617187, 16.137573641461135, 17.80826390009203, 101.34439296756295, 0.0]
[8 5 0 6 3 2 4 1 7]
['Abd Fatty%' 'FEV1%' 'Gender(1=F)' 'FVC % ' 'Sub' 'BMI' 'Vis' 'Age'
 'Disease (CF=1,H=2,copd=3)']


# Muscle

Comb Dia Muscle

In [9]:
predsDiaMCF = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combDiaM.csv', features, 1)
convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combDiaM.csv', predsDiaMCF)

predsDiaMHealthy = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combDiaM.csv', features, 2)
convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combDiaM.csv', predsDiaMHealthy)

predsDiaMCOPD = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combDiaM.csv', features, 3)
convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combDiaM.csv', predsDiaMCOPD)


[115.99192554053283, 57.366474768141295, 59.38989844767796, 63.99707786992012, 40.516757440875395, 48.3708358144619, 47.9914534174915, 58.6182810925753, 0.0]
[8 4 6 5 1 7 2 3 0]
['Dia Muscle V' 'Vis' 'FVC % ' 'FEV1%' 'Age' 'Disease (CF=1,H=2,copd=3)'
 'BMI' 'Sub' 'Gender(1=F)']
[30.061426340190824, 48.63016281110297, 35.905271744429804, 40.747180254043, 35.837312264332255, 37.04049246205861, 30.01509679168624, 30.24285609007544, 0.0]
[8 6 0 7 4 2 5 3 1]
['Dia Muscle V' 'FVC % ' 'Gender(1=F)' 'Disease (CF=1,H=2,copd=3)' 'Vis'
 'BMI' 'FEV1%' 'Sub' 'Age']
[42.06657163380987, 24.71538862758898, 28.08961757982862, 36.14825090829356, 24.13381637718797, 27.663048411100966, 33.45208679045552, 95.1698524722391, 0.0]
[8 4 1 5 2 6 3 0 7]
['Dia Muscle V' 'Vis' 'Age' 'FEV1%' 'BMI' 'FVC % ' 'Sub' 'Gender(1=F)'
 'Disease (CF=1,H=2,copd=3)']


Comb Pecs Muscle

In [10]:
predsPecsMCF = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combPecsM.csv', features, 1)
convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combPecsM.csv', predsPecsMCF)

predsPecsMHealthy = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combPecsM.csv', features, 2)
convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combPecsM.csv', predsPecsMHealthy)

predsPecsMCOPD = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combPecsM.csv', features, 3)
convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combPecsM.csv', predsPecsMCOPD)


[57.7424996626513, 62.692050212427745, 42.596474000027605, 62.15828355967397, 44.854052407618305, 45.376413712666974, 49.682237965848984, 63.032054889215026, 0.0]
[8 2 4 5 6 0 3 1 7]
['Pecs M ' 'BMI' 'Vis' 'FEV1%' 'FVC % ' 'Gender(1=F)' 'Sub' 'Age'
 'Disease (CF=1,H=2,copd=3)']
[40.62713604531786, 36.373106943043354, 32.85531223794909, 31.90855162188341, 32.96656815608536, 31.38229429602735, 35.296557871535626, 33.39535648326442, 0.0]
[8 5 3 2 4 7 6 1 0]
['Pecs M ' 'FEV1%' 'Sub' 'BMI' 'Vis' 'Disease (CF=1,H=2,copd=3)' 'FVC % '
 'Age' 'Gender(1=F)']
[21.914083511726968, 24.905433917381277, 26.147595356009283, 21.09238214439041, 21.247016795203614, 21.041726511444917, 20.650466479233692, 40.788367601723465, 0.0]
[8 6 5 3 4 0 1 2 7]
['Pecs M ' 'FVC % ' 'FEV1%' 'Sub' 'Vis' 'Gender(1=F)' 'Age' 'BMI'
 'Disease (CF=1,H=2,copd=3)']


Comb Para Muscle

In [11]:
predsParaMCF = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combParaM.csv', features, 1)
convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combParaM.csv', predsParaMCF)

predsParaMHealthy = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combParaM.csv', features, 2)
convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combParaM.csv', predsParaMHealthy)

predsParaMCOPD = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combParaM.csv', features, 3)
convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combParaM.csv', predsParaMCOPD)


[44.401748120621384, 21.99700806793793, 25.740734194943787, 22.198882366042206, 21.0954012370127, 23.798844305417582, 23.271462118828204, 25.61151416951684, 0.0]
[8 4 1 3 6 5 7 2 0]
['Para M' 'Vis' 'Age' 'Sub' 'FVC % ' 'FEV1%' 'Disease (CF=1,H=2,copd=3)'
 'BMI' 'Gender(1=F)']
[24.08631604635591, 17.961471024463847, 20.405237531507513, 19.55090997535605, 20.61693383105177, 17.482932465884804, 21.287469107207176, 20.235241668711982, 0.0]
[8 5 1 3 7 2 4 6 0]
['Para M' 'FEV1%' 'Age' 'Sub' 'Disease (CF=1,H=2,copd=3)' 'BMI' 'Vis'
 'FVC % ' 'Gender(1=F)']
[17.531558724983352, 18.95729583479339, 23.675091586450304, 15.78219977430005, 15.947702983686755, 15.794612755442074, 14.528951540566691, 32.308729901033196, 0.0]
[8 6 3 5 4 0 1 2 7]
['Para M' 'FVC % ' 'Sub' 'FEV1%' 'Vis' 'Gender(1=F)' 'Age' 'BMI'
 'Disease (CF=1,H=2,copd=3)']


Comb Abd Muscle

In [12]:
predsAbdMCF = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combAbdM.csv', features, 1)
convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combAbdM.csv', predsAbdMCF)

predsAbdMHealthy = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combAbdM.csv', features, 2)
convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combAbdM.csv', predsAbdMHealthy)

predsAbdMCOPD = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combAbdM.csv', features, 3)
convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combAbdM.csv', predsAbdMCOPD)


[100.82192373933894, 54.987894764167976, 73.75844183129492, 40.542129663884396, 37.77979962929295, 53.04714749291209, 49.45377345903523, 65.29162735735154, 0.0]
[8 4 3 6 5 1 7 2 0]
['Abd M' 'Vis' 'Sub' 'FVC % ' 'FEV1%' 'Age' 'Disease (CF=1,H=2,copd=3)'
 'BMI' 'Gender(1=F)']
[41.25647821020792, 42.5900613692166, 50.257357522097216, 45.23343145448755, 45.66652236760984, 47.72203591679434, 45.598181305613764, 45.23573571644825, 0.0]
[8 0 1 3 7 6 4 5 2]
['Abd M' 'Gender(1=F)' 'Age' 'Sub' 'Disease (CF=1,H=2,copd=3)' 'FVC % '
 'Vis' 'FEV1%' 'BMI']
[16.089286063791203, 32.92183730566178, 15.896662066859523, 25.561615306967948, 28.517617569628275, 22.150047869813925, 20.028873642763934, 28.457109810257077, 0.0]
[8 2 0 6 5 3 7 4 1]
['Abd M' 'BMI' 'Gender(1=F)' 'FVC % ' 'FEV1%' 'Sub'
 'Disease (CF=1,H=2,copd=3)' 'Vis' 'Age']


In [13]:
# create model and fit it
model = RandomForestRegressor()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(y_pred)
print(y_test)
    
print(np.linalg.norm(y_pred - np.array(y_test).astype(float)))


NameError: name 'X_train' is not defined

In [6]:
from sklearn import metrics, svm

clf = svm.SVR()
clf.fit(X_train, y_train)
print("SVR")
print(clf.predict(X_test))
print(y_test)

print(np.linalg.norm(y_pred - np.array(y_test).astype(float)))


NameError: name 'X_train' is not defined